# Load, Subset, and Reproject NISAR GCOV Data with `xarray`

This notebook demonstrates how to load GCOV layers and time series in `xarray` using [utility functions](../util/load_gcov.py) included in this Cookbook

Loading the data as an `xarray.Dataset` provides access to `xarray`’s full toolset and makes it easy to work with lazily loaded, Dask-backed arrays.

<hr>

## Overview

1. [Prerequisites](prereqs)
1. [Find the paths to the GCOV data for the time series](glob)
1. [Load GCOV data into an xarray.Dataset with `load_gcov_ts_xr`](load)
1. [Subset the data](subset)
1. [Reproject the data](reproject)
1. [Calculate statistics and transformations on the data](stats)
1. [Plot the data](plot)
1. [Summary](summary)
1. [Resources and references](resources)

<hr>

(prereqs)=
## 1. Prerequisites
| Prerequisite | Importance | Notes |
| --- | --- | --- |
| [The software environment for this cookbook must be installed](https://github.com/ASFOpenSARlab/NISAR_GCOV_Cookbook/blob/main/notebooks/create_software_environment.ipynb) | Necessary | |

- **Rough Notebook Time Estimate**: 3 minutes

<hr>

(glob)=
## 2. Find the paths to the GCOV data for the time series

In [ ]:
from pathlib import Path

gcov_paths = list(Path("/home/jovyan/NISAR_GCOV_Cookbook/notebooks/time_series_example/data").glob("*.h5"))
gcov_paths

<hr>

(load)=
## 3. Load GCOV data into an xarray.Dataset with `load_gcov_ts_xr`

If you pass a single GCOV path to `load_gcov_ts_xr`, instead of a list of paths, it will create a time series Dataset with a single time step.

`load_gcov_ts_xr` lazily loads all raster data into xarray data structures with delayed HDF5 reads, so the data are not stored in memory until computed.

### 3a. Load a single GCOV dataset

In [ ]:
import sys
from pathlib import Path

util_dir = Path.cwd().parent / "util"
sys.path.insert(0, str(util_dir))

from load_gcov import load_gcov_ts_xr

ds = load_gcov_ts_xr(gcov_paths[0])
ds

#### Access the backscatter raster


In [ ]:
vvvv = ds["VVVV"].isel(time=0, frequency=0)
vvvv

#### With the data in `xarray`, we can view its {abbr}`T (transpose)` attribute without having to first load it into memory 

In [ ]:
# vvvv transpose
vvvv.T

### 3b. Load a GCOV time series

Pass a list of gcov paths instead of a single path.

In [ ]:
ds = load_gcov_ts_xr(gcov_paths)
ds

<hr>

(subset)=
## 4. Subset the data

### 4a. Subset the data at load-time

Load the time series of only the VVVV data for a single frequency in a subset spatial {abbr}`AOI (Area of Interest)`.

In [ ]:
subset_ds = load_gcov_ts_xr(
    gcov_paths, 
    vars_to_load=["VVVV"],
    freqs=["B"],
    y_slice=slice(1900000, 1800000), # (max y, min y)
    x_slice=slice(600000, 700000)) # (min x, max x)

subset_ds

### 4b. Subset already loaded data

In [ ]:
ds = load_gcov_ts_xr(gcov_paths) # load all the data

subset_ds = ds.sel(y=slice(1900000, 1800000), x=slice(600000, 700000)) # create a subset Dataset

subset_ds

<hr>

(reproject)=
## 5. Reproject the data


:::{note} A note about reprojecting with `rioxarray`
`rioxarray` makes it easy to reproject data in an `xarray.Dataset` or `xarray.DataArray`, however it only works with 2D and 3D arrays. The time series Dataset contains 4 dimensions (time, frequency, y, x). 

To reproject, we must select data in 2 or 3 dimensions:
- Select a single frequency and time-step
- Select all (or multiple) time-steps for a single frequency
- Select both frequencies for a single time-step
:::

### 5a. View the source data's CRS

In [ ]:
ds.spatial_ref.attrs["crs_wkt"]

### 5b. Reproject a single frequency and time-step

In [ ]:
# select first time-step and frequency
ds_single_date_freq = ds.isel(time=0, frequency=0)

# reproject to EPSG 4326
ds_single_date_freq = ds_single_date_freq.rio.reproject("EPSG:4326")

ds_single_date_freq 

#### View the updated CRS

In [ ]:
ds_single_date_freq.spatial_ref.attrs["crs_wkt"]

### 5c. Reproject all (or multiple) time-steps for a single frequency

In [ ]:
# select frequency B for all time-steps
ds_single_freq = ds.sel(frequency="B")

# reproject to EPSG 4326
ds_single_freq = ds_single_freq.rio.reproject("EPSG:4326")

ds_single_freq

#### View the updated CRS

In [ ]:
ds_single_freq.spatial_ref.attrs["crs_wkt"]

### 5d. Reproject both frequencies for a single time-step

In [ ]:
# select all frequencies for a single time-step
ds_single_date = ds.isel(time=0)

# reproject to EPSG 4326
ds_single_date = ds_single_date.rio.reproject("EPSG:4326")

ds_single_date

#### View the updated CRS

In [ ]:
ds_single_date.spatial_ref.attrs["crs_wkt"]

<hr>

(stats)=
## 6. Calculate statistics and transformations on the data

### Access a single time step of the VVVV data

In [ ]:
vvvv = ds["VVVV"].sel(time="2025-10-31T04:44:09", frequency="B")
vvvv

### Call the `xarray.mean` function

Notice that you cannot see a mean value. This is because the data is lazily loaded and has not yet been computed or stored in memory.

In [ ]:
vvvv_mean = vvvv.mean()
vvvv_mean

### We can force the value to be computed by casting it to an appropriate datatype or calling `xarray.compute`

Note that computing does not cache the data.

In [ ]:
# cast to float
float(vvvv_mean)

In [ ]:
# call compute
vvvv_mean.compute()

<hr>

(plot)=
## 7. Plot the data

### Convert the data from linear scale (power) to logarithmic scale (dB) for visualization

In [ ]:
import numpy as np

vv_backscatter_power = ds["VVVV"].isel(time=0, frequency=0)
vv_backscatter_dB = 10 * np.log10(vv_backscatter_power)

In [ ]:
float(vv_backscatter_dB.min())

In [ ]:
float(vv_backscatter_dB.max())

### Visualize the log-scale data with `xarray.plot()`

In [ ]:
vv_backscatter_dB.plot(figsize=(14, 10), cmap="gray")

<hr>

(summary)=
## 8. Summary

You have now run an example that loads NISAR GCOV images and time series in `xarray` (along with their metadata), subsets, reprojects the data, and plots an image layer.
<hr>

(resources)=
## 9. Resources and references
- [xarray](https://docs.xarray.dev/en/stable/)

**Author:** Alex Lewandowski